## Menu Q&A Chatbot — Colab Demo

This notebook demonstrates a **hybrid** menu Q&A chatbot:

- **LLM for routing (optional)**: intent + entity extraction
- **Deterministic tools for facts (required)**: prices/calories/categories/discounts are computed from structured data (no hallucinated prices)

### Works without an OpenAI key
If `OPENAI_API_KEY` is **not** set, the system automatically uses the deterministic fallback router.

### Optional OpenAI routing
If you set `OPENAI_API_KEY`, the router will try the LLM path first and fall back automatically on any failure.


In [1]:
# Section 1 — Setup (local + Colab)
#
# By default, this cell sets up imports for *local* Jupyter.
# If you're running in Colab, flip RUN_COLAB_SETUP=True.

import os
import sys
from pathlib import Path

RUN_COLAB_SETUP = False

if RUN_COLAB_SETUP:
    # --- Colab setup ---
    GITHUB_REPO = "https://github.com/andres-troiano/menu-qa-chatbot.git"
    REPO_DIR = "menu-qa-chatbot"

    !git clone $GITHUB_REPO
    %cd $REPO_DIR

    # Install deps into the *current notebook Python* (so imports work)
    !python -m pip -q install -U uv
    !uv pip install --system -e .
else:
    # --- Local Jupyter setup ---
    # Ensure we run from the repo root and can `import src`.
    repo_root = Path.cwd().resolve()
    if repo_root.name == "notebooks":
        repo_root = repo_root.parent

    os.chdir(repo_root)
    if str(repo_root) not in sys.path:
        sys.path.insert(0, str(repo_root))

    print("Notebook Python:", sys.version.split()[0])
    #print("Repo root:", repo_root)


Notebook Python: 3.11.14


In [2]:
# Section 2 — Load the menu index (Bootstrap)
from src.bootstrap import load_index_with_summary

index, summary = load_index_with_summary("data/dataset.json")
summary

{'total_items': 46,
 'total_categories': 7,
 'total_discounts': 9,
 'has_channel_pricing': False,
 'notes': ['No channel-specific price overrides detected in dataset']}

## Section 3 — Run demo questions (NO API key required)

If `OPENAI_API_KEY` is not set, the router will automatically use the deterministic fallback router.


In [3]:
from src.chat import answer

questions = [
    # Price Lookup
    "What is the price of a small NUTTY BOWL?",
    "How much is the ACAI ELIXIR?",
    "How much is the GREEN BOWL Large?",

    # Nutrition Lookup
    "How many calories does the GO GREEN smoothie have?",
    "Calories for Dragon Smoothie",

    # Category Listing
    "Which salads do you have?",
    "What bowls are available?",
    "Show me smoothies",

    # Discount Listing
    "What discounts are available?",
    "Which discounts include coupons?",

    # Discount Trigger Query
    "What items trigger BOGO Any Smoothie discount?",

    # Cross-Channel Price Comparison
    "Is the price for Smoothie - ACAI ELIXIR the same in all channels?",
]

In [5]:
for q in questions:
    print("Q:", q)
    print("A:", answer(q, index))
    print("-" * 80)


Q: What is the price of a small NUTTY BOWL?
A: $11.99 — NUTTY BOWL (Small)
--------------------------------------------------------------------------------
Q: How much is the ACAI ELIXIR?
A: $8.49 — ACAI ELIXIR
--------------------------------------------------------------------------------
Q: How much is the GREEN BOWL Large?
A: $15.99 — GREEN BOWL (Large)
--------------------------------------------------------------------------------
Q: How many calories does the GO GREEN smoothie have?
A: GO GREEN: 240 calories
--------------------------------------------------------------------------------
Q: Calories for Dragon Smoothie
A: DRAGON SMOOTHIE: 250 calories
--------------------------------------------------------------------------------
Q: Which salads do you have?
A: Salads (5 items): Chimichurri Steak & Pot Bowl, Green Glow Bowl, MIGHTY MED SALAD, Power Pesto Chicken Bowl, Supergreen Goddess Salad
--------------------------------------------------------------------------------
Q: Wh

## Section 4 — Optional: Enable OpenAI routing

Optional. If you don’t have an OpenAI key, skip this section.

Set your key like this:


In [4]:
import os

key_present = bool(os.getenv("OPENAI_API_KEY"))
print("OPENAI_API_KEY present:", key_present)

if RUN_COLAB_SETUP and not key_present:
    print(
        "\nYou're in Colab mode and OPENAI_API_KEY isn't set.\n"
        "Paste your key by uncommenting the line below.\n"
    )

# If you want to provide a key manually (Colab), uncomment:
# os.environ["OPENAI_API_KEY"] = "PASTE_YOUR_KEY_HERE"

# Optional: override router model
# os.environ["OPENAI_MODEL"] = "gpt-4o-mini"


OPENAI_API_KEY present: True


## Section 5 — Confirm router mode (debug)

You should see `meta.router` as:
- `"fallback"` if no key is set
- `"llm"` if a key is set and the LLM path succeeds


In [5]:
from src.router import route

rr = route("What is the price of a small NUTTY BOWL?")
rr

RouteResult(route=RouterOutput(intent='get_price', item='NUTTY BOWL', portion='small', category=None, discount=None, channel=None), meta=RouteMeta(router='llm', reason=None, model='gpt-4o-mini', error_type=None, error_message=None), raw_llm_output=None)

## Section 6 — LLM-powered routing examples (optional)

If you enabled `OPENAI_API_KEY`, these queries will typically route via the LLM.

Important behavior:
- If the dataset does not contain channel pricing overrides, the answer should be honest (not invented).
- If discount triggers can’t be fully joined, the answer should be best-effort + explain the limitation.


In [6]:
for q in questions:
    rr = route(q)
    print(q, "=>", rr.meta)
    
    print("Q:", q)
    print("A:", answer(q, index))
    print("-" * 80)


What is the price of a small NUTTY BOWL? => router='llm' reason=None model='gpt-4o-mini' error_type=None error_message=None
Q: What is the price of a small NUTTY BOWL?
A: $11.99 — NUTTY BOWL (Small)
--------------------------------------------------------------------------------
How much is the ACAI ELIXIR? => router='llm' reason=None model='gpt-4o-mini' error_type=None error_message=None
Q: How much is the ACAI ELIXIR?
A: $8.49 — ACAI ELIXIR
--------------------------------------------------------------------------------
How much is the GREEN BOWL Large? => router='llm' reason=None model='gpt-4o-mini' error_type=None error_message=None
Q: How much is the GREEN BOWL Large?
A: $15.99 — GREEN BOWL (Large)
--------------------------------------------------------------------------------
How many calories does the GO GREEN smoothie have? => router='llm' reason=None model='gpt-4o-mini' error_type=None error_message=None
Q: How many calories does the GO GREEN smoothie have?
A: GO GREEN: 240 c

## Section 7 — Optional: Interactive mini chat

Run this cell to ask questions interactively.


In [ ]:
while True:
    q = input("Ask a menu question (or 'exit'): ").strip()
    if q.lower() in {"exit", "quit"}:
        break
    print(answer(q, index))


GO GREEN: 240 calories
Salads (5 items): Chimichurri Steak & Pot Bowl, Green Glow Bowl, MIGHTY MED SALAD, Power Pesto Chicken Bowl, Supergreen Goddess Salad
